IF-B Kecerdasan Buatan <br>
KELOMPOK 2: <br>
JONATHAN NATANNAEL ZEFANYA <br>
DAFFA DANINDRA <br>
IKHWAN EL FARIS <br>
DETHALISA AULIA KIRANA

In [1]:
# Mengimpor pustaka yang diperlukan
import csv
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Mengimpor modul files dari google.colab untuk memungkinkan pengunggahan file
from google.colab import files
uploaded = files.upload()  # Upload CSV file

Saving csv_KNN_Liver_CSU_1.csv to csv_KNN_Liver_CSU_1.csv


In [2]:
# Step 1: mengakses data
def load_data(file):
    data = []
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Skip the header ('ALB','ALP','ALT','AST','BIL','PROT','GGT')
    for row in csv_reader:
        data.append(row)
    return header, data

In [3]:
# Step 2: melakukan preprocessing data (mengubah ke numerik dan menangani nilai yang hilang jika diperlukan)
def preprocess_data(data):
    # Convert semua data ke float, kecuali kolom terakhir (Label)
    for i in range(len(data)):
        data[i] = [float(x) if j < len(data[i])-1 else x for j, x in enumerate(data[i])]
    return data

In [4]:
# Load dan preprocess data
filename = list(uploaded.keys())[0]
with open(filename, 'r') as file:
  header, data = load_data(file)  # Load the data

data = preprocess_data(data)

In [6]:
# Cetak semua data
print('Overall Data:')
for row in data:
    print(row)

Overall Data:
[38.5, 52.5, 7.7, 22.1, 7.5, 69.0, 12.1, 'Blood Donor']
[38.5, 70.3, 18.0, 24.7, 3.9, 76.5, 15.6, 'Blood Donor']
[46.9, 74.7, 36.2, 52.6, 6.1, 79.3, 33.2, 'Blood Donor']
[43.2, 52.0, 30.6, 22.6, 18.9, 75.7, 33.8, 'Blood Donor']
[47.0, 19.1, 38.9, 164.2, 17.0, 70.1, 90.4, 'Hepatitis']
[42.0, 38.2, 63.3, 187.7, 14.0, 70.5, 40.2, 'Hepatitis']
[45.0, 27.5, 10.5, 37.8, 10.0, 74.5, 35.9, 'Hepatitis']
[45.0, 34.6, 44.3, 96.2, 16.0, 82.1, 48.1, 'Hepatitis']
[41.0, 34.4, 12.1, 60.9, 6.0, 71.1, 33.1, 'Hepatitis']
[41.0, 43.1, 2.4, 83.5, 6.0, 66.5, 130.0, 'Fibrosis']
[39.0, 43.1, 23.8, 114.7, 11.0, 81.3, 127.3, 'Fibrosis']
[45.0, 26.9, 23.1, 125.0, 17.0, 73.0, 72.2, 'Fibrosis']
[49.0, 45.2, 19.3, 69.1, 30.0, 72.3, 28.4, 'Fibrosis']
[34.0, 137.8, 4.8, 35.6, 9.0, 57.5, 133.4, 'Cirrhosis']
[31.0, 85.3, 4.8, 60.2, 200.0, 71.8, 151.0, 'Cirrhosis']
[36.0, 69.6, 14.9, 263.1, 40.0, 68.6, 61.0, 'Cirrhosis']
[29.0, 11.3, 7.1, 101.9, 31.0, 70.0, 65.6, 'Cirrhosis']


In [7]:
# Step 3: memisahkan data menjadi data (X) dan label (y)
def split_features_labels(data):
    X = [row[:-1] for row in data]
    y = [row[-1] for row in data]
    return X, y

In [17]:
 # Memisahkan menjadi data (X) dan label (y)
X, y = split_features_labels(data)

# Membagi data training dan data testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print("Label testing: ")
print(y_test)
print("Data testing: ")
print(X_test)

Label testing: 
['Blood Donor', 'Blood Donor', 'Hepatitis', 'Cirrhosis', 'Fibrosis']
Data testing: 
[[38.5, 52.5, 7.7, 22.1, 7.5, 69.0, 12.1], [38.5, 70.3, 18.0, 24.7, 3.9, 76.5, 15.6], [42.0, 38.2, 63.3, 187.7, 14.0, 70.5, 40.2], [36.0, 69.6, 14.9, 263.1, 40.0, 68.6, 61.0], [45.0, 26.9, 23.1, 125.0, 17.0, 73.0, 72.2]]


In [18]:
# Step 4: menghitung jarak Euclidean
def euclidean_distance(point1, point2):
    # |jarak data testing - jarak data latih|
    distance = np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))
    return distance

In [19]:
# Step 5: mengimplementasikan algoritma KNN
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    distances_list = []
    for test_point in X_test:
        # Menghitung jarak dari test_point ke semua titik pelatihan
        distances = [(euclidean_distance(test_point, train_point), label) for train_point, label in zip(X_train, y_train)]
        distances_list.append(sorted(distances, key=lambda x: x[0]))  # Simpan jarak untuk output dan urutkan dari jarak terdekat ke terjauh
        # Mengurutkan berdasarkan jarak dan mendapatkan k label terdekat
        k_nearest_labels = [label for _, label in sorted(distances)[:k]]
        # Mendapatkan label yang paling umum (pemungutan suara mayoritas)
        most_common_label = Counter(k_nearest_labels).most_common(1)[0][0]
        predictions.append(most_common_label)
    return predictions, distances_list

In [20]:
 # Menetapkan nilai k
k = 3  # isi K bebas untuk kNN (k dari tetangga terdekat)

# Membuat Label Prediksi
y_pred, distances_list = knn_predict(X_train, y_train, X_test, k)

# Menampilkan jarak antara data pelatihan dan pengujian
print('Jarak data training dan data testing:')
for i, distances in enumerate(distances_list):
    print(f'Data Testing ke-{i+1}:')
    for distance, label in distances:
        print(f'Jarak: {distance:.2f}, Label: {label}')

Jarak data training dan data testing:
Data Testing ke-1:
Jarak: 34.54, Label: Blood Donor
Jarak: 39.05, Label: Hepatitis
Jarak: 48.02, Label: Hepatitis
Jarak: 53.47, Label: Blood Donor
Jarak: 57.36, Label: Fibrosis
Jarak: 93.45, Label: Hepatitis
Jarak: 107.57, Label: Cirrhosis
Jarak: 133.42, Label: Fibrosis
Jarak: 149.45, Label: Cirrhosis
Jarak: 149.52, Label: Fibrosis
Jarak: 169.04, Label: Hepatitis
Jarak: 242.79, Label: Cirrhosis
Data Testing ke-2:
Jarak: 32.82, Label: Blood Donor
Jarak: 39.01, Label: Blood Donor
Jarak: 50.55, Label: Hepatitis
Jarak: 54.59, Label: Hepatitis
Jarak: 59.80, Label: Fibrosis
Jarak: 91.40, Label: Hepatitis
Jarak: 113.70, Label: Cirrhosis
Jarak: 132.81, Label: Fibrosis
Jarak: 138.32, Label: Cirrhosis
Jarak: 146.37, Label: Fibrosis
Jarak: 168.52, Label: Hepatitis
Jarak: 241.92, Label: Cirrhosis
Data Testing ke-3:
Jarak: 63.77, Label: Hepatitis
Jarak: 94.64, Label: Hepatitis
Jarak: 111.12, Label: Cirrhosis
Jarak: 120.97, Label: Fibrosis
Jarak: 128.45, Label: 

In [21]:
# Accuracy Score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 40.00%


In [22]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[1 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
